In [ ]:
class CurrentBoard:
  
  
  def __init__(self,setup_of_board = ['wwww','wwww','wwww','    ','    ','bbbb', 'bbbb', 'bbbb'], newEvaluation = None):
      self.board = setup_of_board
      if newEvaluation != None:
        self.state_of_play = newEvaluation
      self.state, self.evaluation  = self.state_of_play(self.board)

  def new_board(self,new_setup):
    self.board = new_setup
    self.state,self.evaluation = self.state_of_play(self.board)


  def display_board(self):
    print("-----------------")
    for row in range(8):
      c = self.board[row]
      if (row % 2) == 0:
        print("| |{}| |{}| |{}| |{}|".format(c[0], c[1],c[2],c[3]))
      else: 
        print("|{}| |{}| |{}| |{}| |".format(c[0], c[1],c[2],c[3]))
      print("-----------------")
    



  def adjacent_squares(self, position, inc = 999):
    # inc = 1 for whites, inc = -1 for black, inc  =0 for kings
    # return a list of all possible positions reachable in 1 move from position

    if inc  > 10:
      return self.adjacent_squares(position,inc = self.directionAt(position))
    if inc == 0:
        return self.adjacent_squares(position, inc = -1) + self.adjacent_squares(position,inc = 1)

    all_moves = []
    start_row = position[0]
    start_i = position[1]

    new_row = start_row+inc
    if (new_row>=0) and (new_row < 8):
      all_moves.append((new_row,start_i))
      if (start_row % 2) == 0:
        if (start_i<3):
          all_moves.append((new_row, start_i+1))
      else:
        if (start_i >0):
          all_moves.append((new_row, start_i-1))       

    return all_moves

  def playingAs(self,piece):
    if piece == 'W':
      return 'w'
    if piece == "B":
      return 'b'
    return piece


  def all_possible_moves_for(self,playing_colour): # all moves for "w" or "b"
    non_kill_boards = []
    kill_boards = []
    found_kill = False
    for row in range(8):
      for i in range(4):
        if self.playingAs(self.get_piece_at((row,i))) == playing_colour:
          new_non_kills, new_kills = self.construct_boards_for((row,i),found_kill)
          if len(new_kills) > 0:
            found_kill = True
            kill_boards += new_kills
          else:
            non_kill_boards += new_non_kills
    if found_kill:
      return kill_boards
    else:
      return non_kill_boards



  def get_piece_at(self, position):
    if position != None:
      return self.board[position[0]][position[1]]
    else:
      return None

  def row_for(self,position):
    return position[0]

  def i_for(self,position):
    return position[1]


  def jump_to(self, from_position, over_position):
    row_dir = self.row_for(over_position) - self.row_for(from_position)
    new_row = self.row_for(over_position) + row_dir
    if (new_row <0) or (new_row>7):
      return None
    if (new_row % 2) == 0:
      if (self.i_for(from_position) == self.i_for(over_position)):
        new_i = self.i_for(from_position) - 1
      else:
        new_i = self.i_for(from_position)  +1
    else:

      if (self.i_for(from_position) == self.i_for(over_position)):
        new_i = self.i_for(from_position) + 1
      else:
        new_i = self.i_for(from_position)  -1   
    if (new_i >=0) and (new_i<4):
      return (new_row,new_i)

    
  def all_kills(self,start_position, current_position,kill_list):


    all_kills_from_here = self.get_adjacent_kills(current_position, kill_list)
    all_completed_moves = []
    if (all_kills_from_here == []):
      if (kill_list == []):
        return []
      else:
        return [self.board_for(start_position, current_position,kill_list)]
    
    for kill in all_kills_from_here:
      all_completed_moves += self.all_kills(start_position,self.jump_to(current_position, kill), kill_list + [kill])

    return all_completed_moves

  def king_me(self,piece):
    if piece == 'w':
      return 'W'
    if piece == 'b':
      return 'B'
    return piece
  
  def board_for(self, from_position, to_position, kill_list):
    board_def = []
    remove_list = kill_list +[from_position]
  
    piece = self.get_piece_at(from_position)
    if (self.row_for(to_position) %7 ) == 0:
      piece = self.king_me(piece)
    for row_index in range(8):
      row = self.board[row_index]
      for empty_square in remove_list:
        if row_index == self.row_for(empty_square):
          row =  self.replace_at(row, self.i_for(empty_square) ,' ' ) 
      if row_index == self.row_for(to_position):

        row = self.replace_at(row, self.i_for(to_position) , piece )
      
      board_def.append(row)
 
    return CurrentBoard(board_def,newEvaluation=self.state_of_play)



  def move_piece_to(self, from_position, to_position):

    return self.board_for(from_position,to_position,[from_position])


  def get_adjacent_kills(self,position,kill_list):
      piece  = self.get_piece_at(position)
      kills = []
      all_adjacent_squares = self.adjacent_squares(position)
      for square in all_adjacent_squares:
        if self.is_opponent_of(piece,self.get_piece_at(square)):
          if self.is_not_already_killed(square,kill_list) and self.get_piece_at( self.jump_to(position,square) ) == ' ':
            kills.append(square)
      return kills

  def is_not_already_killed(self,square, list_of_positions):
    return not ( square in list_of_positions)



  def is_opponent_of(self,piece1,piece2):
    return  (self.playingAs(piece1) != self.playingAs(piece2)) and (piece1 != ' ') and (piece2 != ' ')




  def replace_at(self,row_as_string, index, character):
    return row_as_string[:index] + character + row_as_string[index+1:]

  def construct_boards_for(self,  from_position , found_kill):
    non_kill_boards = []
    kill_boards = []

    kill_boards += self.all_kills(from_position,from_position,[])
    if not found_kill  and len(kill_boards)==0:
      to_position_list = self.adjacent_squares(from_position)
      for new_position in to_position_list:
        if self.get_piece_at(new_position) == " ":
          non_kill_boards.append(self.move_piece_to(from_position, new_position))

    return non_kill_boards, kill_boards


  def directionAt(self,position):
    piece = self.get_piece_at(position)
    color = self.playingAs(piece)
    inc  = 0 
    if piece == color:
      if piece == 'w':
        inc = 1
      else:
        inc = -1
    return inc



    
  






  



In [ ]:
class Search_Tree_Node:
  def __init__(self, game , playing_char, ply = 0, max_ply = 6):
    self.children =[]
    self.ply_depth = ply
    self.max_ply = max_ply
    self.value_is_assigned = False
    self.value = 999
    self.board_for_node = game
    self.board_eval = game.evaluation if playing_char == "w" else -game.evaluation
    self.playing = playing_char
    if self.board_for_node.state == "U":
      self.generate_children()
    else:
      # game is won by either black or white
      nodePlayingSameasAI = (self.ply_depth %2 == 0)

      if nodePlayingSameasAI:
        if self.playing == self.board_for_node.state:
          self.value = 1000
        else:
          self.value = -1000

      else:
        if self.playing == self.board_for_node.state:
          self.value = -1000
        else:
          self.value = 1000

      self.value_is_assigned = True
      



  def max_min_value(self):
    # if no value is assigned we will assign the max(or min) of the value of the children

    if  self.value_is_assigned:
        return self.value

    self.children = sorted(self.children, key= lambda x: x.max_min_value() )
    self.value_is_assigned = True
    if (self.ply_depth % 2) == 0:
      self.value = self.children[-1].value
    else:
      self.value = self.children[0].value
    return self.value


  def other_char(self,current):
    if current == "w":
      return "b"
    if current == "b":
      return "w"
    return " "

  def generate_children(self):
    if self.max_ply > self.ply_depth:
      for next_move in self.board_for_node.all_possible_moves_for(self.playing):
        self.children.append(Search_Tree_Node(next_move, self.other_char(self.playing), self.ply_depth + 1))
      if len(self.children) == 0:
        self.value_is_assigned = True
        if (self.ply_depth % 2) == 0:
          self.value = -1000000  
        else:
          self.value = 1000000
    
    
    else:
      self.value = self.board_eval
      self.value_is_assigned = True


In [ ]:
def versusBots(evalf1, evalf2):
  max_ply = 6
  V1MoveNext = False;
  V1isPlaying ='b'
  print("First Eval function is playing " + V1isPlaying)
  V2isPlaying = 'w'

  if V1MoveNext:
    currentBoard = CurrentBoard(newEvaluation=evalf1)
  else:
    currentBoard = CurrentBoard(newEvaluation=evalf2)

  currentGameOver = False


  while not currentGameOver:
    if V1MoveNext:
      st = Search_Tree_Node(currentBoard,V1isPlaying, max_ply=max_ply)
      st.max_min_value()
      if len(st.children) == 0:
        currentGameOver = True
        currentBoard.state = V2isPlaying
      else: 
        currentBoard = CurrentBoard(st.children[-1].board_for_node.board, evalf2)
    

    else:  # V2 is playing next
      st = Search_Tree_Node(currentBoard,V2isPlaying, max_ply=max_ply)
      st.max_min_value()
      if len(st.children) == 0:
        currentGameOver = True
        currentBoard.state = V1isPlaying
      else:
        currentBoard =  CurrentBoard(st.children[-1].board_for_node.board, evalf1)
    
    
    currentGameOver = currentBoard.state != "U"
    
    print("The evaluation of the boards below is " + str(currentBoard.evaluation))
    currentBoard.display_board()
    V1MoveNext = not V1MoveNext
    if max_ply<8:
      count = 0
      for r in currentBoard.board:
        count+= r.count(' ')

      if count > 20:
        max_ply = 15;


  print("The winner is " +str(currentBoard.state))


In [ ]:

def state_of_play(boardAsString):

  whites = 0
  blacks = 0
  for row in boardAsString:
    whites += row.count('w') + 2* row.count('W')
    blacks += row.count('b') + 2* row.count('B')
  
  evaluation = whites - blacks
  if whites == 0 :
    return 'b', evaluation
  if blacks == 0:
    return 'w', evaluation
  return 'U', evaluation




def state_of_playV2(boardAsString):
    pieceValue = 1
    kingMultiplier = 2.5


    whites = 0
    blacks = 0

    for row in boardAsString:
      whites += row.count('w')+ kingMultiplier* row.count('W')
      blacks += row.count('b')+ kingMultiplier*  row.count('B')
    
    evaluation = whites - blacks
    if whites == 0 :
      return 'b',evaluation
    if blacks == 0:
      return 'w',evaluation
    return 'U',evaluation

def state_of_playV3(game):
    player1 = 'w'
    player2 = 'b'

    material_advantage_weight=1.0
    king_advantage_weight=2.0
    mobility_advantage_weight=60.0
    edge_space_advantage_weight=0.5

    piece_count = {"w": 0, "b": 0}
    king_count = {"w": 0, "b": 0}
    mobility_count = {"w": 0, "b": 0}
    edge_space_count = {"w": 0, "b": 0}
    combined_advantage = 0.0

    for row in range(8):
        for col in range(4):
            piece = game[row][col]
            if piece != " ":
                piece_count[piece.lower()] += 1
                if piece == "B" or piece == "W":
                    king_count[piece.lower()] += 1
                    if piece == player1.upper():
                        if row >= 2 and row <= 5 and col >= 1 and col <= 2:
                            # give bonus for kings in the middle 4 rows and middle 2 columns
                            combined_advantage += 0.1
                        elif row >= 1 and row <= 6 and col >= 0 and col <= 3:
                            # give bonus for kings away from the edges
                            combined_advantage += 0.05
                    #Then it is the enimies pieces
                    else:
                        if row >= 2 and row <= 5 and col >= 1 and col <= 2:
                            # give bonus for kings in the middle 4 rows and middle 2 columns
                            combined_advantage -= 0.1
                        elif row >= 1 and row <= 6 and col >= 0 and col <= 3:
                            # give bonus for kings away from the edges
                            combined_advantage -= 0.05
                else:
                    mobility_count[piece] += len(get_legal_moves(game, row, col))
                    #small piece edge advantage
                    if piece == player1:
                        if row == 0 or row == 8 or col == 0 or col == 3:
                            # give bonus for pieces on the edges
                            edge_space_count[player1] += 1
                    else:
                        if row == 0 or row == 8 or col == 0 or col == 3:
                            # give bonus for pieces on the edges
                            edge_space_count[player2] += 1

    # material advantage
    material_advantage = ((piece_count[player1] - piece_count[player2]) / material_advantage_weight)

    # king advantage
    king_advantage = ((king_count[player1] - king_count[player2]) / king_advantage_weight)

    # mobility advantage
    mobility_advantage = ((mobility_count[player1] - mobility_count[player2]) / mobility_advantage_weight)

    # edge space advantage
    edge_space_advantage = ((edge_space_count[player1] - edge_space_count[player2]) / edge_space_advantage_weight)

    # combine advantages
    combined_advantage += material_advantage + king_advantage + mobility_advantage
    
    if piece_count["w"] == 0 and king_count["w"]==0:
        return 'b', combined_advantage
    if piece_count["b"] == 0 and king_count["b"]==0:
        return 'w', combined_advantage
    
    return 'U', combined_advantage
  

In [ ]:
def get_legal_moves(game, row, col):
  #print("original row : "+str(row))
  #print("original col : "+str(col))
  piece = game[row][col]
  legal_moves = []
    
  is_king = piece == "B" or piece == "W"
    
  for i, j in [(-1, -1), (-1, 1), (1, -1), (1, 1)]:
      new_row, new_col = row + i, col + j
      # print("new row : "+str(new_row))
      # print("new col : "+str(new_col))
      if not is_king and (new_row < 0 or new_row >= 8):
          # print("here")
          continue
      if new_col < 0 or new_col >= 4:
          continue
      if game[new_row][new_col] == " ":
          legal_moves.append((new_row, new_col))
      elif game[new_row][new_col].lower() != piece.lower():
          jump_row, jump_col = new_row + i, new_col + j
          if not is_king and (jump_row < 0 or jump_row >= 8):
              continue
          if jump_col < 0 or jump_col >= 4:
              continue
          if game[jump_row][jump_col] == " ":
              legal_moves.append((jump_row, jump_col))
                
  return legal_moves

In [ ]:
versusBots(state_of_playV3,state_of_play)